# Spatial and spectral resolutions.

* **Special requirements:** A Google account, access to Google Earth Engine.
* **Prerequisites:** You should have completed the `2.1_ENGN3903_Satellite images and bands`, the `2.2 ENGN3903_Images, collections, and filters` notebook.


## Background

> Note: Refer to Lecture 2 of week 2 for the relevant content for this lab

Resolution is the ability to discriminate information in an image. The discrimination of information refers not only to the spatial detail (spatial resolution) but also to the number of spectral wavebands, their bandwidths, and spectral cover (spectral resolution), the temporal frequency of observations (temporal resolution) and the signal to noise ratio or its ability to distinguish variations in the energy detected (radiometric resolution). One may also have to consider the sensor's potential to acquire information at different viewing angles (angular resolution) or polarization channels (polimetric resolution).


## Aims of the practical session

This practical has three aims:
1. To understand the differences between the spatial and spectral resolutions of different satellite sensors.
2. To plot the spectral signature of different land cover types, and
3. Learn how to use band-combinations to highlight different landscape features 

***

## Description

In this notebook we'll load images from three different sensors and will use them to understand the differences between spatial and spectral resolution in the remote sensing context. 

First we will:
- Load Landsat, Sentinel 2, and MODIS images for the Canberra region.

Then we will:
- visualize the images of the different sensors side-by-side to understand the concept of spatial resolution;
- Compare the spectral resolution of different sensors (how many 'bands' or 'measurements' thye have);
- Apply various band-combinations to Landsat images to highlight different landscape features.

<div class="alert alert-block alert-warning">
<b>Assessment:</b> Once you finish the practical and the excercises, remember to submit your notebook through Wattle by Sunday.
Challenges are optional and will not be part of the assessment.
</div>

***

## Getting started


### Load packages

Import Python packages that are used for the analysis.


In [ ]:
# %matplotlib inline
import os
import ee
import numpy as np
import pandas as pd
import geemap as gmap
import matplotlib.pyplot as plt

### Connect to Google Earth Engine (GEE)

Connect to the GEE so we can access GEE datasets and computing assets.
You may be required to input your Google account name and password. Please keep those safe and don't share them with anyone.

In [ ]:
m = gmap.Map()

***

## Comparing the spatial resolution of different sensors.

Load the `MODIS` and `Sentinel-2` image collections that intersect the ACT and visualize them in a map

First, let's create a polygon around the ACT

In [ ]:
# create a polygon around the ACT
act = ee.Geometry.Polygon([[148.7392751586051,-36.011462319908816],
   [149.8598806273551,-36.011462319908816],
   [149.8598806273551,-35.1087997777942],
   [148.7392751586051,-35.1087997777942],
   [148.7392751586051,-36.011462319908816]])

# Let's add the polygon to the map
Map = gmap.Map(center=[-35.2041, 149.2721], zoom=10)
Map.addLayer(act,{},'act')

### A quick note on image composites
In the code below, we filter the datasets to a time-range `.filterDate('2019-02', '2019-04')`, this will give us a two-month long time-series of satellite images over that region. We then _reduce_ that time-series (collapse the time-dimension) with the function `.median()`, which will create what Earth Observation scientists call a 'composite image', that is, an image that is representative of all images over a given time. Individual remote sensing images can be affected by noisy data, including clouds, cloud shadows, and haze. To produce cleaner images that can be compared more easily across time, we can create 'summary' images or 'composites' that combine multiple images into one. Median summaries are an extremely useful and very commonly used approach to summarising time-series of satellite images.

> Note, to 'clip' an imageCollection to a polygon, we can't rely on the syntax `.filterBounds()` like when we clip just an Image, instead we need to map the `image.clip` function over the time-series. ie. `modis.map(lambda image: image.clip(polygon))`

In [ ]:
# Now let's add the image collections.
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
modis = ee.ImageCollection("MODIS/061/MYD09A1").select([
    'sur_refl_b01',
    'sur_refl_b02',
    'sur_refl_b03',
    'sur_refl_b04',
    'sur_refl_b05',
    'sur_refl_b06',
    'sur_refl_b07'])

# Filter the image collections by date, and by location, then reduce time dimension using a median
s2 = s2.filterDate('2019-02', '2019-04').map(lambda image: image.clip(act)).median()
modis = modis.filterDate('2019-02', '2019-04').map(lambda image: image.clip(act)).median()

# Lastly, let's give some visualization paramaters to each collection.
s2VisParam = {'bands': ["B4","B3","B2"],
              'max': 2700, #this data hasn't been rescaled
              'min': 0}

# Note that the visualization are different for each image collection.
modisVisParams = {'bands': ["sur_refl_b01","sur_refl_b04","sur_refl_b03"],
              'max': 2100,
              'min': 0}

#add images to the map
Map.addLayer(s2, s2VisParam, 's2')
Map.addLayer(modis, modisVisParams, 'modis')
Map.addLayerControl()
Map

<div class="alert alert-block alert-danger">
    
### Exercise 1 - Add a `Landsat 8` image collection to the map.

You've done this in earlier labs, just make sure you filter the image collection with the same dates as the ones above, and reduce the time-series using the `.median()` function.  Remember to rescale the dataset as well, and call your rescaling function `rescale_landsatC2()`. For this one example, you can ignore cloud masking.

</div>  

In [ ]:
# Your code goes here



<div class="alert alert-block alert-danger">

### Exercise 2 - Understanding spatial resolution

**Answer the following questions:**

1) What is the smallest feature you can identify using the MODIS, Landsat, and Sentinel 2 imagery?
    
2) Which satellite sensor has the highest and lowest spatial resolution.



To help with your ansers, try identifying the following features / places / landmarks:
>- Telstra Tower
>- a car
>- a football/cricket field
>- Canberra Airport
>- Lake George,
>- a crop field
>- a mountain range or a large forest.

</div>    

Your answer goes here:

.

***

## Comparing the spectral resolution of different sensors

There are only a few hyperspectral sensors in orbit.  Fortunately, we have access to [Hyperion](https://www.usgs.gov/centers/eros/science/earth-observing-1-eo-1) data. Hyperion was a hyperspectral sensor that was Decommissioned in 2017, but gathered data in many places around the globe.

As with the other sensors, we can add these data to our map, and filter by date and location.


> **Caution** Because Hyperion only provides irradiance data, we might have to 1) either perform atmospheric correction, or 2) load TOA Landsat data and uncorrected MODIS so the plots are comparable. We aren't doing this here, but its something to keep in mind when considering the differences between sensors.

> Remember, whenever we load Landsat Collection2, we need to rescale the digital numbers to surface-reflectance. You should have defined a function for this in Exercise 1.

In [ ]:
Map2 = gmap.Map(center=[-35.2041, 149.2721], zoom=9)

# Get the Hyperion image collection and clip to the canberra region
hyper = ee.ImageCollection("EO1/HYPERION").map(lambda image: image.clip(act)).median()

# load Landsat but only load the optical bands an then rescale
clearC2 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_090085_20210118') \
        .select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']).clip(act)

clearC2 = rescale_landsatC2(clearC2)

# Set the visualization parameters for the images we're going to display
hyperVisParams = {'bands': ["B035","B023","B015"],
                 'min':500,
                 'max':6000}

landsatC2_vis = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
              'min': 0, # this data has been rescaled so is 0-1
              'max': 0.4}

# Add the layers to the map
Map2.addLayer(modis, modisVisParams, 'MODIS image collection')
Map2.addLayer(clearC2, landsatC2_vis,' Landsat image')
Map2.addLayer(hyper, hyperVisParams, 'Hyperion hyperspectral image collection')
Map2.set_plot_options(add_marker_cluster=True)


### Displaying the spectral signatures of different land covers

Let's use the plotting tool to  produce simple spectral signature plots for the different sensors (we did this last week, but we manually enetered the numbers). Click on the map toolbar and select the 'Plotting' icon as shown below. **Pay attention to the images selected in the toolbar for this exercise**.

![3.1_fig3.PNG](https://github.com/nicolasyounes/engn3903/raw/main/figures/3.1_fig3.PNG)

Now click anywhere on the image and look at the spectral signature of the selected feature. Is it what you would expect?

In [ ]:
Map2

<div class="alert alert-block alert-danger">
    
### Exercise 3 - Understanding the spectral resolution.

**Compare and contrast:**

1) The spatial *coverage* of the MODIS data, versus the Hyperion data
    
2) The spectral resolution of the MODIS, Landsat, and Hyperion images.

</div>    


Your answers go here

.

If we wanted to, we can export these points and their corresponding band values to a CSV and Shapefile to use them in another software (e.g. QGIS/ArcGIS). 

**Below, change the path to the your folder on the NCI, and your plots in the map above will be exported as .csv files. The points your select on the map will also be exported as shapefile (we'll discuss vector file formats more soon).**

In [ ]:
# Change the path to the your folder on the NCI, 
folder = '/scratch/du53/path-to-a-folder/'
Map2.extract_values_to_points(os.path.join(folder, 'spectral_signatures.shp'))

### Extracting the spectral signatures of different land covers programmatically

Some points covering different landcover classes have been collected for you. We will use these points to extract the spectral signature of the landcover classes programmatically.

The web app : [geojson.io](http://geojson.io/#map=2/0/20) is great for quickly collecting geometries interactively.


**First we need to load the points into the map**


In [ ]:
#### load pre-collected samples:
path = "https://raw.githubusercontent.com/nicolasyounes/engn3903/main/figures/landcover_points.geojson"
points = gmap.geojson_to_ee(path)

# Convert to dataframe so we can see the values
df = gmap.ee_to_df(points, remove_geom=True)
df.head()

Now recreate the map with all the satellite images on it, and let's add the points as well

In [ ]:
Map3 = gmap.Map(center=[-35.9659, 149.4965], zoom=9)

# Add the layers to the map
Map3.addLayer(modis, modisVisParams, 'MODIS image collection')
Map3.addLayer(clearC2, landsatC2_vis,' Landsat image')
Map3.addLayer(hyper, hyperVisParams, 'Hyperion hyperspectral image collection')

#plot the points
Map3.addLayer(points, {}, 'points to extract')

#mapping options
Map3.addLayerControl()
Map3.set_plot_options(add_marker_cluster=True)
Map3

### Extract band values for each pixel in the samples collected

Below, we use the function `'sampleRegions'` to extract the pixel values at the location identified by the `points` object.

In [ ]:
pointsLandsat = clearC2.sampleRegions(**
                               {'collection': points,
                                'scale': 30,
                                'geometries': True,
                               'tileScale': 8}
                               )

# Now let's do the same with the MODIS images
pointsModis = modis.sampleRegions(**
                               {'collection': points,
                                'scale': 500,
                                'geometries': True,
                               'tileScale': 8}
                               )
# Now let's do the same with the Hyperion images
pointsHyperion = hyper.sampleRegions(**
                               {'collection': points,
                                'scale': 30,
                                'geometries': True,
                               'tileScale': 8}
                               )


Now each point has the values for each Landsat band, MODIS and Hyperion band, we can convert the points with the band values to a 'dataframe' (a table)

In [ ]:
df_landsat = gmap.ee_to_df(pointsLandsat, remove_geom=True)
df_modis = gmap.ee_to_df(pointsModis, remove_geom=True)
df_hyperion = gmap.ee_to_df(pointsHyperion, remove_geom=True)

# Now we'll sort the 'label' column to make the figure below look a bit better.
df_landsat = df_landsat.sort_values(by=['label'])
df_modis = df_modis.sort_values(by=['label'])
df_hyperion = df_hyperion.sort_values(by=['label'])

# and we re-oder the MODIS columns. See https://lpdaac.usgs.gov/products/myd09a1v061/ to understand the order.
modisColumnOrder = ['landcover', 'label', 'sur_refl_b03','sur_refl_b04','sur_refl_b01','sur_refl_b02',
                                          'sur_refl_b05','sur_refl_b06','sur_refl_b07']
df_modis = df_modis[modisColumnOrder]

We can print one of the dataframe to see how its organised

In [ ]:
df_modis

### Plot the spectral signatures of each sensor

In [ ]:
with plt.style.context('ggplot'): #use ggplot styles (similar to R)
    #set the plot mosaic grid so its neat.
    fig, axes = plt.subplot_mosaic(
        [['a)', 'b)'], 
         ['c)', 'c)'],
    ], constrained_layout=True, figsize=(15,7))
    
    #do some data wrangling to get the dataframe in a form that plots nicely
    df_modis.reset_index(drop=True).set_index('label').drop('landcover', axis=1).T.sort_index(axis=0).groupby(level=0, axis=1).mean().plot(ax=axes['a)'])
    df_landsat.reset_index(drop=True).set_index('label').drop('landcover', axis=1).T.sort_index(axis=0).groupby(level=0, axis=1).mean().plot(ax=axes['b)'])
    df_hyperion.reset_index(drop=True).set_index('label').drop('landcover', axis=1).T.sort_index(axis=0).groupby(level=0, axis=1).mean().plot(ax=axes['c)'])
    
    # Set the lables for the axes    
    axes['a)'].set_ylabel('MODIS reflectance')
    axes['b)'].set_ylabel('Landsat reflectance')
    axes['c)'].set_ylabel('Hyperion radiance')
    axes['a)'].set_xlabel('MODIS bands')
    axes['b)'].set_xlabel('Landsat bands')
    axes['c)'].set_xlabel('Hyperion bands')
    axes['a)'].set_title('MODIS data')
    axes['b)'].set_title('Landsat data')
    axes['c)'].set_title('Hyperion data');


***

## Band Combinations

The previous exercise showed you that features in an image look different in different bands. Some bands are better to detect the presence (or absence) of a feature.
For example, the Near Infrared (NIR) and Short-Wave Infrared (SWIR) bands are very useful to detect vegetation, and water. We can combine these bands to create 'color composites' (a.k.a false color images). A false color image is used to reveal or enhance features otherwise invisible or poorly visible to a human eye.

![false%20color.png](https://github.com/nicolasyounes/engn3903/raw/main/figures/false%20color.png)


In [ ]:
Map4 = gmap.Map(center=[-35.9659, 149.4965], zoom=8)

# Get  a Landsat image to display
clearC2 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_090085_20210118') \
        .select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7',])

#rescale the image
clearC2 = rescale_landsatC2(clearC2)

# Select the bands we want to display for the false color composite
falseColor_vis = {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'min':0, 'max':0.4}
trueColor_vis = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min': 0,'max': 0.4}


# Add the layers to the map
Map4.addLayer(clearC2,  falseColor_vis, "False Color NIR,R,G")
Map4.addLayer(clearC2, trueColor_vis,'True Colour R,G,B')
Map4.addLayerControl()
Map4

<div class="alert alert-block alert-danger">

### Exercise 3 -  False colour images

It is your turn to create some different false colour images.

Using the Landsat 8 band designations here to help you: https://www.usgs.gov/landsat-missions/landsat-8,

**Answer the following questions:**
1. How does dense vegetation appear in the false colour band combination: [NIR, SWIR1, Red]? Why?
2. How does bare soil appear in the false colour band combination: [SWIR1, NIR, Red]? Why? 
3. How does shallow, sediment filled water appear in the false colour band combination: [SWIR1, NIR, Green]? Why? Try comparing Lake George (or a farm dam) with Lake Burley Griffin.

</div>

**The answers to the questions go here.**


.

***

## Summary

In this notebook you have learned about how different sensors have different spatial and spectral resolutions.  The attributes of specific satellite sensors **will** affect your analysis, so you'll have to understand which sensor is better for which analysis.  Lastly, we also considered how different band combinations can be used to visualise features in the landscape. 

***

***

## Additional information

**Sources:** 

**License:** The code in this notebook is licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle (**check**) site 

**Last modified:** August 2024

***